In [1]:
import pandas as pd
import numpy as np
import json
from glob import glob

In [2]:
arr_iupac_name = []
with open('database/Dict_segment.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_iupac_name = np.concatenate((arr_iupac_name, d[key]))

arr_cid = []
with open('database/Dict_segment_cid.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_cid = np.concatenate((arr_cid, d[key]))

arr_iupac_cut = []
with open('database/Dict_segment_cut.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_iupac_cut = np.concatenate((arr_iupac_cut, d[key]))

In [3]:
import os
dir_iupac_name = 'database/iupac_name/*.json'
dir_iupac_cut = 'database/iupac_cut/*.json'
dir_cid = 'database/cid/*.json'

for file in sorted(glob(dir_iupac_name)):
  print(file)
  with open(file, 'r') as f:
    d = json.load(f)
  arr_iupac_name = np.concatenate((arr_iupac_name, d['values']))
print(arr_iupac_name.shape, arr_iupac_name[-2:])
print('-'*20)

for file in sorted(glob(dir_iupac_cut)):
  print(file)
  with open(file, 'r') as f:
    d = json.load(f)
  arr_iupac_cut = np.concatenate((arr_iupac_cut, d['values']))
print(arr_iupac_cut.shape, arr_iupac_cut[-2:])
print('-'*20)

for file in sorted(glob(dir_cid)):
  print(file)
  with open(file, 'r') as f:
    d = json.load(f)
  arr_cid = np.concatenate((arr_cid, d['values']))
print(arr_cid.shape, arr_cid[-2:])

database/iupac_name/Dict_ester_iupac_1-5-21.json
database/iupac_name/Dict_segment_12-2-20.json
database/iupac_name/Dict_segment_12-7-20.json
(302119,) ['(2S)-2-[(4-hydroxyphenyl)methyl]-3-methylidenepent-4-enoic acid'
 '[4-(carboxymethyl)phenyl]oxidanium']
--------------------
database/iupac_cut/Dict_ester_cut_1-5-21.json
database/iupac_cut/Dict_segment_cut_12-2-20.json
database/iupac_cut/Dict_segment_cut_12-7-20.json
(302119,) ['(|2|S|)|-|2|-|[|(|4|-|hydroxy|phen|yl|)|meth|yl|]|-|3|-|meth|yl|idene|pent|-|4|-|en|oic acid'
 '[|4|-|(|carboxy|meth|yl|)|phen|yl|]|oxidanium']
--------------------
database/cid/Dict_ester_cid_1-5-21.json
database/cid/Dict_segment_cid_12-2-20.json
database/cid/Dict_segment_cid_12-7-20.json
(302119,) ['154546822' '154550731']


In [4]:
print(arr_iupac_name[339])
print(arr_iupac_cut[339])
print(arr_cid[339])

2,5,7-trimethyloctan-3-ol
2|,|5|,|7|-|tri|meth|yl|oct|an|-|3|-|ol
22990510


In [5]:
n_file = 1

f = open(f'cid/cid{n_file}.txt', 'w')
for i in range(1, len(arr_cid)+1):
    if (i-1)%50000 == 0 and i != 1:
        f.close()
        n_file += 1
        f = open(f'cid/cid{n_file}.txt', 'w')

    f.write(str(arr_cid[i-1]))
    f.write('\n')
f.close()

In [6]:
df = pd.DataFrame(list(zip(arr_cid, arr_iupac_name, arr_iupac_cut)), columns=['cid', 'iupacname', 'label'])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

302119
301978


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301978 entries, 0 to 302118
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   cid        301978 non-null  object
 1   iupacname  301978 non-null  object
 2   label      301978 non-null  object
dtypes: object(3)
memory usage: 9.2+ MB


In [8]:
df.tail()

,cid,iupacname,label
302114,154262256,1-(4-hydroxyphenyl)heptan-3-one,1|-|(|4|-|hydroxy|phen|yl|)|hept|an|-|3|-|one
302115,154265009,3-(4-hydroxyphenyl)-2-silyloxypropanoate,3|-|(|4|-|hydroxy|phen|yl|)|-|2|-|sil|yl|oxy|p...
302116,154364721,8-(4-hydroxyphenyl)octanoate,8|-|(|4|-|hydroxy|phen|yl|)|oct|an|oate
302117,154546822,(2S)-2-[(4-hydroxyphenyl)methyl]-3-methylidene...,(|2|S|)|-|2|-|[|(|4|-|hydroxy|phen|yl|)|meth|y...
302118,154550731,[4-(carboxymethyl)phenyl]oxidanium,[|4|-|(|carboxy|meth|yl|)|phen|yl|]|oxidanium


In [9]:
df.head()

,cid,iupacname,label
0,297,methane,meth|ane
1,6324,ethane,eth|ane
2,6334,propane,prop|ane
3,7843,butane,but|ane
4,8003,pentane,pent|ane


In [10]:
from sklearn.model_selection import train_test_split

train_df, all_val_df = train_test_split(df, test_size = 0.3, shuffle = True)

val_df, test_df = train_test_split(all_val_df, test_size = 0.5, shuffle = True)

# Train 70, Val 15, Test 15

In [11]:
print(len(train_df), len(val_df), len(test_df))

211384 45297 45297


In [12]:
train_df.reset_index(inplace=True)
val_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [13]:
train_df['file_names'] = train_df['cid'].astype(str)+'.png'
val_df['file_names'] = val_df['cid'].astype(str)+'.png'
test_df['file_names'] = test_df['cid'].astype(str)+'.png'

In [14]:
train_df.to_csv('dataframe/df_train.csv', index=False)
val_df.to_csv('dataframe/df_val.csv', index=False)
test_df.to_csv('dataframe/df_test.csv', index=False)

In [15]:
 np.savetxt("cid.txt", arr_cid, fmt="%s")

In [18]:
pd.read_csv('dataframe/df_train.csv').tail()

,index,cid,iupacname,label,file_names
211379,266225,20483615,ethyl 1-prop-2-enylaziridine-2-carboxylate,eth|yl| |1|-|prop|-|2|-|en|yl|aziridine|-|2|-|...,20483615.png
211380,195037,101792263,S-phenyl (2S)-2-formamido-3-methylbutanethioate,S|-|phen|yl| |(|2S|)|-|2|-|forma|mido|-|3|-|me...,101792263.png
211381,201929,102067424,ethyl (E)-8-chlorooct-7-en-5-ynoate,eth|yl| |(|E|)|-|8|-|chloro|oct|-|7|-|en|-|5|-...,102067424.png
211382,111234,71527260,[(E)-4-ethylhex-2-enyl] (4-methoxyphenyl) carb...,[|(|E|)|-|4|-|eth|yl|hex|-|2|-|en|yl|]| |(|4|-...,71527260.png
211383,109741,91704775,"1-O-pentyl 6-O-(2,2,2-trichloroethyl) hexanedi...","1|-|O|-|pent|yl| |6|-|O|-|(|2|,|2|,|2|-|tri|ch...",91704775.png


# Make Tokenize Label


In [2]:
train_df = pd.read_csv("dataframe/df_train.csv")
val_df = pd.read_csv("dataframe/df_val.csv")
test_df = pd.read_csv("dataframe/df_test.csv")

In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
from nami.AI.kme_tokenize import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['iupacname'].values)

100%|██████████| 211384/211384 [1:54:45<00:00, 30.70it/s]


## Save tokenizer state

In [5]:
import json

with open('../Image_Captioning/tokenizer/index2word.json', 'w') as f1:
  json.dump(tokenizer.index2word, f1)

with open('../Image_Captioning/tokenizer/longest_sentences.json', 'w') as f2:
  json.dump(tokenizer.longest_sentences, f2)

with open('../Image_Captioning/tokenizer/num_words.json', 'w') as f3:
  json.dump(tokenizer.num_words, f3)

with open('../Image_Captioning/tokenizer/word2count.json', 'w') as f4:
  json.dump(tokenizer.word2count, f4)

with open('../Image_Captioning/tokenizer/word2index.json', 'w') as f5:
  json.dump(tokenizer.word2index, f5)

In [6]:
seq_train = tokenizer.text_to_sequences(train_df['iupacname'].values)
seq_val = tokenizer.text_to_sequences(val_df['iupacname'].values)
seq_test = tokenizer.text_to_sequences(test_df['iupacname'].values)

100%|██████████| 45297/45297 [23:22<00:00, 32.29it/s]


In [7]:
seq_train.shape

(211384, 95)

In [8]:
new_col = [f'label{i}' for i in range(1, seq_train.shape[1]+1)]

seq_train_df = pd.DataFrame(seq_train, columns=new_col)
seq_val_df = pd.DataFrame(seq_val, columns=new_col)
seq_test_df = pd.DataFrame(seq_test, columns=new_col)

seq_val_df.head()

,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,...,label86,label87,label88,label89,label90,label91,label92,label93,label94,label95
0,1,30,8,9,14,4,82,34,17,82,...,0,0,0,0,0,0,0,0,0,0
1,1,79,5,18,8,9,4,5,27,5,...,0,0,0,0,0,0,0,0,0,0
2,1,7,8,9,17,5,30,8,5,4,...,0,0,0,0,0,0,0,0,0,0
3,1,30,8,9,14,49,16,5,4,5,...,0,0,0,0,0,0,0,0,0,0
4,1,18,64,5,4,5,8,9,4,5,...,0,0,0,0,0,0,0,0,0,0


In [9]:
train_df_concat = pd.concat([train_df.drop(['index', 'label'], axis=1), seq_train_df], axis=1)
val_df_concat = pd.concat([val_df.drop(['index', 'label'], axis=1), seq_val_df], axis=1)
test_df_concat = pd.concat([test_df.drop(['index', 'label'], axis=1), seq_test_df], axis=1)

In [10]:
val_df_concat.head()

,cid,iupacname,file_names,label1,label2,label3,label4,label5,label6,label7,...,label86,label87,label88,label89,label90,label91,label92,label93,label94,label95
0,102476271,"methyl (2R,3R)-2-amino-3-hydroxy-3-(3-methylph...",102476271.png,1,30,8,9,14,4,82,...,0,0,0,0,0,0,0,0,0,0
1,57375822,tert-butyl 2-amino-3-pyridin-3-ylpropanoate,57375822.png,1,79,5,18,8,9,4,...,0,0,0,0,0,0,0,0,0,0
2,16099663,ethyl 3-methyl-2-oxocyclohex-3-ene-1-carboxylate,16099663.png,1,7,8,9,17,5,30,...,0,0,0,0,0,0,0,0,0,0
3,93972463,methyl (2S)-2-hydroxybut-3-enoate,93972463.png,1,30,8,9,14,49,16,...,0,0,0,0,0,0,0,0,0,0
4,85736,butan-2-yl 2-cyanoacetate,85736.png,1,18,64,5,4,5,8,...,0,0,0,0,0,0,0,0,0,0


In [11]:
test_df_concat.head()

,cid,iupacname,file_names,label1,label2,label3,label4,label5,label6,label7,...,label86,label87,label88,label89,label90,label91,label92,label93,label94,label95
0,86636857,(3-methyloxan-4-yl) methanesulfonate,86636857.png,1,14,17,5,30,8,52,...,0,0,0,0,0,0,0,0,0,0
1,102037167,"methyl (2R,4R)-2-cyano-4-phenyl-2-propan-2-ylp...",102037167.png,1,30,8,9,14,4,82,...,0,0,0,0,0,0,0,0,0,0
2,24813332,tert-butyl N-hydroxy-N-(3-methyl-1-thiophen-2-...,24813332.png,1,79,5,18,8,9,80,...,0,0,0,0,0,0,0,0,0,0
3,22093490,methyl 4-fluoro-2-methoxy-5-morpholin-4-ylbenz...,22093490.png,1,30,8,9,26,5,6,...,0,0,0,0,0,0,0,0,0,0
4,100974435,[(3S)-hex-1-en-3-yl] methyl carbonate,100974435.png,1,29,14,84,16,5,72,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train_df_concat.to_csv('data/processed/df_train_label.csv', index=False)
val_df_concat.to_csv('data/processed/df_val_label.csv', index=False)
test_df_concat.to_csv('data/processed/df_test_label.csv', index=False)